In [0]:
import pyspark
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('Spam Detection').getOrCreate()

In [0]:
#df = spark.read.csv('/FileStore/tables/SMSSpamCollectio',inferSchema = True, header = True,sep='\t')
df = sql("SELECT * FROM smsspamcollection_1") #from databreaks table.
df.show()

+-----+--------------------+
Class| Text|
+-----+--------------------+
 ham|Go until jurong p...|
 ham|Ok lar... Joking ...|
 spam|Free entry in 2 a...|
 ham|U dun say so earl...|
 ham|Nah I don't think...|
 spam|FreeMsg Hey there...|
 ham|Even my brother i...|
 ham|As per your reque...|
 spam|WINNER!! As a val...|
 spam|Had your mobile 1...|
 ham|I'm gonna be home...|
 spam|SIX chances to wi...|
 spam|URGENT! You have ...|
 ham|I've been searchi...|
 ham|I HAVE A DATE ON ...|
 spam|XXXMobileMovieClu...|
 ham|Oh k...i'm watchi...|
 ham|Eh u remember how...|
 ham|Fine if thats th...|
 spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows

In [0]:
from pyspark.sql.functions import length

In [0]:
df = df.withColumn('length',length(df['text']))

In [0]:
df.show()

+-----+--------------------+------+
Class| Text|length|
+-----+--------------------+------+
 ham|Go until jurong p...| 111|
 ham|Ok lar... Joking ...| 29|
 spam|Free entry in 2 a...| 155|
 ham|U dun say so earl...| 49|
 ham|Nah I don't think...| 61|
 spam|FreeMsg Hey there...| 147|
 ham|Even my brother i...| 77|
 ham|As per your reque...| 160|
 spam|WINNER!! As a val...| 157|
 spam|Had your mobile 1...| 154|
 ham|I'm gonna be home...| 109|
 spam|SIX chances to wi...| 136|
 spam|URGENT! You have ...| 155|
 ham|I've been searchi...| 196|
 ham|I HAVE A DATE ON ...| 35|
 spam|XXXMobileMovieClu...| 149|
 ham|Oh k...i'm watchi...| 26|
 ham|Eh u remember how...| 81|
 ham|Fine if thats th...| 56|
 spam|England v Macedon...| 155|
+-----+--------------------+------+
only showing top 20 rows

In [0]:
#avg length of ham and spam messages
df.groupby('class').mean().show()

+-----+-----------------+
class| avg(length)|
+-----+-----------------+
 ham| 71.4545266210897|
 spam|138.6706827309237|
+-----+-----------------+

In [0]:
from pyspark.ml.feature import (Tokenizer, StopWordsRemover, CountVectorizer,
                                IDF, StringIndexer)

In [0]:
#data prepareation
tokenizer = Tokenizer(inputCol='Text',outputCol='token_text')
stop_remove = StopWordsRemover(inputCol = 'token_text',outputCol ='stop_token')
count_vec = CountVectorizer(inputCol = 'stop_token',outputCol = 'c_vec')
idf = IDF(inputCol = 'c_vec',outputCol = 'tf_idf')
ham_spam_to_numaric = StringIndexer(inputCol='Class',outputCol='label')


In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [0]:
from pyspark.ml.classification import NaiveBayes

In [0]:
nb = NaiveBayes()

In [0]:
from pyspark.ml import Pipeline

In [0]:
#pipeline
data_prep_pipeline = Pipeline(stages=[ham_spam_to_numaric,tokenizer,
                                      stop_remove,count_vec,idf,clean_up])

In [0]:
cleaner = data_prep_pipeline.fit(df)

In [0]:
clean_data= cleaner.transform(df)

In [0]:
cleaner_data.columns

Out[107]: ['label']

In [0]:
clean_data = clean_data.select('label','features')

In [0]:
clean_data.show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(13424,[7,11,31,6...|
 0.0|(13424,[0,24,297,...|
 1.0|(13424,[2,13,19,3...|
 0.0|(13424,[0,70,80,1...|
 0.0|(13424,[36,134,31...|
 1.0|(13424,[10,60,139...|
 0.0|(13424,[10,53,103...|
 0.0|(13424,[125,184,4...|
 1.0|(13424,[1,47,118,...|
 1.0|(13424,[0,1,13,27...|
 0.0|(13424,[18,43,120...|
 1.0|(13424,[8,17,37,8...|
 1.0|(13424,[13,30,47,...|
 0.0|(13424,[39,96,217...|
 0.0|(13424,[552,1697,...|
 1.0|(13424,[30,109,11...|
 0.0|(13424,[82,214,47...|
 0.0|(13424,[0,2,49,13...|
 0.0|(13424,[0,74,105,...|
 1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows

In [0]:
training,test = clean_data.randomSplit([0.7,0.3])

In [0]:
spam_detector = nb.fit(training)

In [0]:
df.printSchema()

root
-- Class: string (nullable = true)
-- Text: string (nullable = true)
-- length: integer (nullable = true)

In [0]:
test_results = spam_detector.transform(test)

In [0]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(13424,[0,1,3,9,1...|[-573.18788685208...|[1.0,3.9740447493...| 0.0|
 0.0|(13424,[0,1,7,8,1...|[-1160.3044057456...|[1.0,3.8953487364...| 0.0|
 0.0|(13424,[0,1,9,14,...|[-540.23885114090...|[1.0,1.0071738025...| 0.0|
 0.0|(13424,[0,1,14,18...|[-1379.5231941504...|[1.0,2.9861522454...| 0.0|
 0.0|(13424,[0,1,21,27...|[-770.35563353291...|[1.0,4.8607344700...| 0.0|
 0.0|(13424,[0,1,21,27...|[-1015.1887453164...|[1.0,3.3119007658...| 0.0|
 0.0|(13424,[0,1,72,10...|[-666.15586818472...|[1.0,1.0933072830...| 0.0|
 0.0|(13424,[0,1,3657,...|[-127.88322653713...|[0.99997852623920...| 0.0|
 0.0|(13424,[0,2,3,6,9...|[-3297.2754467640...|[1.0,1.3025757673...| 0.0|
 0.0|(13424,[0,2,3,8,2...|[-1619.3310797447...|[1.0,1.4407827684...| 0.0|
 0.0|(13424,[0,2,4,5,1...|[-2720.2707483163...|[1.0,9.0864968690...| 0.0|
 0.0|(13424,[0,2,4,5,1...|[-2720.2707483163...|[1.0,9.0864968690...| 0.0|
 0.0|(13424,[0,2,4,8,1...|[-1311.2489357265...|[1.0,1.7699751002...| 0.0|
 0.0|(13424,[0,2,4,10,...|[-1239.5971958727...|[1.0,3.3643171748...| 0.0|
 0.0|(13424,[0,2,4,11,...|[-1128.5399071640...|[1.0,4.6895591757...| 0.0|
 0.0|(13424,[0,2,4,44,...|[-1900.1598317518...|[1.0,3.2956315174...| 0.0|
 0.0|(13424,[0,2,4,128...|[-640.59502664343...|[1.0,6.0848174556...| 0.0|
 0.0|(13424,[0,2,7,8,1...|[-467.89665950725...|[0.99999999598712...| 0.0|
 0.0|(13424,[0,2,7,11,...|[-728.82108568474...|[1.0,1.5602938750...| 0.0|
 0.0|(13424,[0,2,7,11,...|[-1414.8785201724...|[1.0,8.3328099207...| 0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [0]:
#label vs prediction by multiclass evaluation metrix
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
acc_evel = MulticlassClassificationEvaluator()

In [0]:
acc = acc_evel.evaluate(test_results)

In [0]:
print('ACC of NB Model')
print(acc)

ACC of NB Model
0.9277129216584169

In [0]:
#From above acc value we can conclude that the accurary of above NB model is 92%.